In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics_causal_model

In [3]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [4]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

In [5]:
from Metrics import rpf1a_from_tp_fp_tn_fn
from collections import defaultdict

def tally_counts(r, filter):
    tally = defaultdict(int)
    for k,v in r.items():
        if filter(k):
            for prop in "tp,tn,fp,fn".split(","):
                tally[prop] += v[prop]
    return tally

def get_causal_relation_metrics(collection, params, include_concept_codes=True):
    dicts = []
    for r in db[collection].find({}):
        d = {}
        cr_counts = tally_counts(r, lambda c: "->" in c)
        (rec, p, cr_f1, a) = rpf1a_from_tp_fp_tn_fn(cr_counts["tp"],cr_counts["fp"],cr_counts["tn"],cr_counts["fn"])
        d["cr_micro_f1"] = cr_f1
        d["cr_micro_rec"]  = rec
        d["cr_micro_prec"] = p
        if include_concept_codes:
            concept_counts = tally_counts(r, lambda c: c[0].isdigit())
            (rec, p, concept_f1, a) = rpf1a_from_tp_fp_tn_fn(concept_counts["tp"],concept_counts["fp"],concept_counts["tn"],concept_counts["fn"])
            d["concept_micro_f1"] = concept_f1
            d["concept_micro_rec"]  = rec
            d["concept_micro_prec"] = p
        parms = r["parameters"]
        for p in params:
            d[p] = parms[p]
        dicts.append(d)
    df = pd.DataFrame(dicts)
    fields = ("cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec," + ",".join(params)).split(",")
    if not include_concept_codes:
        fields = [f for f in fields if "concept" not in f]
    return df[fields].sort_values("cr_micro_f1", ascending=False)

In [6]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

# Feature Selection on Shift Reduce Parser

In [7]:
# REMOVE the non generic parameters, like window size
def get_df_sorted_by_f1score_generic(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "feats":          "$parameters.extractors",
            
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision" ] + params
        return df[cols]
    return df

### Top Result (Best VD Accuracy

#### CB

In [8]:
col = ["CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD"]

rows = []
for coll in col:
    df = get_df_sorted_by_f1score_generic(coll, "")
    dct = df.iloc[0].to_dict()
    dct["Algo"] = coll
    rows.append(dct)

df=pd.DataFrame(rows)
df.sort_values("micro_f1_score", ascending=False)

,Algo,micro_f1_score,micro_precision,micro_recall
0,CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARA...,0.748462,0.791057,0.710219


#### SC

In [9]:
col = ["CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD"]

rows = []
for coll in col:
    df = get_df_sorted_by_f1score_generic(coll, "")
    dct = df.iloc[0].to_dict()
    dct["Algo"] = coll
    rows.append(dct)

df=pd.DataFrame(rows)
df.sort_values("micro_f1_score", ascending=False)

,Algo,micro_f1_score,micro_precision,micro_recall
0,CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARA...,0.813308,0.820764,0.805985


## Results Sorted by Hyper Params

#### CB

In [10]:
def extract_parameter(s, param_name):
    s = s.replace("("," ").replace(")"," ")
    keys = s.split(" ")
    return [(key,val.replace(",","").replace("'","")) for key,val in [k.split("=") for k in keys if "=" in k] if key == param_name][0][-1]

extract_c_val = lambda s: extract_parameter(s, "C")
extract_penalty_val = lambda s: extract_parameter(s, "penalty")
extract_dual_val = lambda s: extract_parameter(s, "dual")

s = "LogisticRegression(C=0.1, class_weight=None, dual=True, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='liblinear', tol=0.0001, verbose=0, warm_start=False)"
extract_c_val(s)

'0.1'

In [11]:
col = ["CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD"]

df_feat_sel = get_df_sorted_by_f1score_generic(col[0], "algorithm,beta,max_epochs,ngrams,stemmed,num_feats_MEAN", filter_cols=True) 
df_feat_sel["C"] = df_feat_sel["algorithm"].apply(extract_c_val)
df_feat_sel["penalty"] = df_feat_sel["algorithm"].apply(extract_penalty_val)
df_feat_sel["dual"] = df_feat_sel["algorithm"].apply(extract_dual_val)
del df_feat_sel["algorithm"]
#df = df_feat_sel.head(20).style.set_properties(subset=['algorithm'], **{'width': '500px'})
df_feat_sel.head(10)

,micro_f1_score,micro_recall,micro_precision,beta,max_epochs,ngrams,stemmed,num_feats_MEAN,C,penalty,dual
0,0.748462,0.710219,0.791057,0.50,10,1,True,94761.2,0.1,l2,True
1,0.748237,0.716423,0.783008,0.30,3,1,True,82068.4,0.1,l2,False
2,0.747829,0.722993,0.774433,1.00,2,1,True,69890.2,0.1,l2,False
3,0.747763,0.716788,0.781536,0.30,3,1,True,82070.2,0.1,l2,True
4,0.747547,0.722993,0.773828,1.00,2,1,True,69893.4,0.1,l2,True
5,0.747506,0.710949,0.788026,0.10,10,1,True,96958.0,0.1,l2,True
6,0.747241,0.741241,0.753338,0.75,2,1,True,69665.8,0.1,l2,False
7,0.746788,0.721168,0.774295,0.10,3,1,True,78832.8,0.1,l2,True
8,0.746742,0.721533,0.773777,0.75,2,1,True,60324.4,1.0,l2,True
9,0.746538,0.708394,0.789024,0.50,5,1,True,89913.0,0.1,l2,False


#### SC

In [12]:
col = ["CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM2_VD"]

df_feat_sel = get_df_sorted_by_f1score_generic(col[0], "algorithm,beta,max_epochs,ngrams,stemmed,num_feats_MEAN", filter_cols=True) 
df_feat_sel["C"] = df_feat_sel["algorithm"].apply(extract_c_val)
df_feat_sel["penalty"] = df_feat_sel["algorithm"].apply(extract_penalty_val)
df_feat_sel["dual"] = df_feat_sel["algorithm"].apply(extract_dual_val)
del df_feat_sel["algorithm"]
#df = df_feat_sel.head(20).style.set_properties(subset=['algorithm'], **{'width': '500px'})
df_feat_sel.head(10)

,micro_f1_score,micro_recall,micro_precision,beta,max_epochs,ngrams,stemmed,num_feats_MEAN,C,penalty,dual
0,0.813308,0.805985,0.820764,1.00,3,1,True,55036.8,0.5,l2,False
1,0.811732,0.803703,0.819922,1.00,3,1,True,55045.6,0.5,l2,True
2,0.811680,0.800152,0.823545,0.75,3,1,True,54970.0,0.5,l2,True
3,0.811068,0.799138,0.823360,0.10,5,1,True,59600.8,0.5,l2,False
4,0.810478,0.800406,0.820806,0.50,5,1,True,61283.2,0.5,l2,True
5,0.810360,0.801420,0.819502,0.40,5,1,True,61643.2,0.5,l2,False
6,0.809903,0.800659,0.819362,0.10,5,1,True,59994.4,0.5,l2,True
7,0.809805,0.800152,0.819693,0.75,5,1,True,59742.2,0.5,l2,True
8,0.809609,0.803449,0.815864,0.20,5,1,True,70620.8,0.5,l1,False
9,0.809596,0.808775,0.810419,0.75,3,1,True,59622.2,0.1,l2,False


In [13]:
# Got here without error
print("Success")

Success
